In [1]:
import cl_graph_bert as cgm
import torch
from torch import nn
import json
from transformers import BertModel, BertConfig, BertTokenizer, AdamW
import tqdm

from torch.utils.data import Dataset, DataLoader
# '''
# class ReviewDataset(torch.utils.data.Dataset):
#     def __init__(self, tokens, labels, ids):
#         self.tokens = tokens
#         self.labels = labels
#         self.ids = ids
        
#     def __len__(self):
#         return len(self.tokens['input_ids'])

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.tokens.items()}
#         out = {"tokens": item, "label": self.labels[idx], "id": self.ids[idx]}
#         return out

# def process_text(filepath, batch_size):
#     reviews = []
#     data = open(filepath)
#     for line in data.readlines():
#         reviews.append(json.loads(line))

#     review_texts = []
#     review_scores = []

#     for sample in reviews:
#         if 'reviewText' in sample and 'overall' in sample:
#             review_texts.append(sample['reviewText'])
#             if sample['overall'] >= 4:
#                 review_scores.append(1)
#             else:
#                 review_scores.append(0)
                
#     train_reviews = review_texts[:len(review_texts)//2]
#     train_ids = [i for i in range(0, len(review_texts)//2+1)]
#     test_reviews = review_texts[len(review_texts)//2:]
#     test_ids = [i for i in range(len(review_texts)//2, len(review_texts))]
#     train_scores = review_scores[:len(review_texts)//2]
#     test_scores = review_scores[len(review_texts)//2:]

#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#     tokenized_train_reviews = tokenizer(train_reviews, return_tensors="pt", padding='max_length', truncation=True)
#     tokenized_test_reviews = tokenizer(test_reviews, return_tensors="pt", padding='max_length', truncation=True)

#     train_dataset = ReviewDataset(tokenized_train_reviews, train_scores, train_ids)
#     test_dataset = ReviewDataset(tokenized_test_reviews, test_scores, test_ids)

#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
#     return train_loader, test_loader
# '''

In [2]:
import dgl
g = dgl.load_graphs("./graphs/industrial_and_scientific_5_core_new.dgl")[0][0]

train_nids = g.ndata["train_mask"]
test_nids = g.ndata['test_mask']

sampler = dgl.dataloading.NeighborSampler([5, 10])

In [3]:
import math
num_reviews = g.num_nodes("Review")
# '''
# class IDDataset(torch.utils.data.IterableDataset):
#     def __init__(self, start, end):
#         super(IDDataset).__init__()
#         assert end > start, "this example code only works with end >= start"
#         self.start = start
#         self.end = end

#     def __iter__(self):
#         worker_info = torch.utils.data.get_worker_info()
#         if worker_info is None:  # single-process data loading, return the full iterator
#             iter_start = self.start
#             iter_end = self.end
#         else:  # in a worker process
#             # split workload
#             per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
#             worker_id = worker_info.id
#             iter_start = self.start + worker_id * per_worker
#             iter_end = min(iter_start + per_worker, self.end)
#         return iter(range(iter_start, iter_end))

# class IDDataset(torch.utils.data.Dataset):
#     def __init__(self, ids):
#         self.ids = ids
        
#     def __len__(self):
#         return len(self.ids)

#     def __getitem__(self, idx):
#         out = {"id": self.ids[idx]}
#         return out

# train_ds = IDDataset(torch.tensor([i for i in range(0, num_reviews//2+1)]))
# test_ds = IDDataset(torch.tensor([i for i in range(num_reviews//2, num_reviews)]))
# '''

In [4]:
train_nids = {}
test_nids = {}
for ntype in g.ntypes:
    if ntype != "Review":
        pass
    else:
        train_nids[ntype] = g.nodes(ntype)[g.ndata["train_mask"]["Review"] == 1]
        test_nids[ntype] = g.nodes(ntype)[g.ndata["test_mask"]["Review"] == 1]

In [5]:
batch_size = 2
'''
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size)
'''

train_loader = dgl.dataloading.DataLoader(
    g, train_nids, sampler,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=0)

test_loader = dgl.dataloading.DataLoader(
    g, test_nids, sampler,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=0)

In [6]:
model = cgm.CLIPGraphModel(
    rel_types = g.etypes,
    emb_types = {x: g.number_of_nodes(x) for x in g.ntypes},
    device="cuda"
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
device = 'cuda'
model.to(device)
model.graph_model.load_state_dict(torch.load("./pretrained_statedict_0.02531265653669834.pt", map_location=torch.device(device)))
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([{"params":model.language_model.parameters(),"lr": 0.00001},
                              {"params":model.graph_model.parameters(), "lr": 0.00001},
                              {"params":model.language_projection.parameters(), "lr": 0.001},
                              {"params":model.graph_projection.parameters(), "lr": 0.001}])

In [8]:
epochs = 2

model.train()

for epoch in range(epochs):
    epoch_loss = 0
    for batch, labels, blocks in tqdm.tqdm(train_loader):            
        optimizer.zero_grad()
        print(batch)
        blocks = [b.to(device) for b in blocks]
        loss = model(g.to(device), "Review", batch["Review"].to(device), blocks)["loss"]    
        epoch_loss += loss
        
        loss.backward()
        optimizer.step()
    epoch_loss /= len(train_loader)
    val_loss = 0
    with torch.no_grad():
        for batch, labels, blocks in tqdm.tqdm(test_loader):
            blocks = [b.to(device) for b in blocks]
            loss = model(g.to(device), "Review", batch["Review"].to(device), blocks)["loss"]

            val_loss += loss
    val_loss /= len(test_loader)
    print("End of Epoch", epoch)
    print("Training loss:", epoch_loss)
    print("Test loss:", val_loss)

  0%|                                                                                       | 0/19265 [00:00<?, ?it/s]

{'Brand': tensor([760, 185]), 'Customer': tensor([6040, 1339]), 'Product': tensor([2205,  390]), 'Review': tensor([33263,  6005, 33252, 33284, 33270, 33199, 33186,  6014,  6024,  6036,
         6031,  6012, 11494, 27289, 32651, 37533, 59434,  3344,  4024,  4085])}


  0%|                                                                                       | 0/19265 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 60.00 MiB (GPU 0; 23.69 GiB total capacity; 16.11 GiB already allocated; 30.56 MiB free; 16.21 GiB reserved in total by PyTorch)

In [11]:
class BertGraphMLP(nn.Module):
    def __init__(self, model, encoder="language_emb", num_labels=2, finetune=False):
        super(BertGraphMLP, self).__init__()
        self.num_labels = num_labels
        self.BertGraph = model
        self.encoder = encoder
        self.classifier = nn.Linear(512, num_labels)
        self.finetune = finetune
       
        if self.finetune:
            self.BertGraph.eval()

    def forward(self, g, ids):
        if self.finetune:
            out = self.BertGraph(g, "Review", ids)
        else:
            with torch.no_grad():
                out = self.BertGraph(g, "Review", ids)
        if self.encoder == "language_emb":
            embs = out["language_emb"]
        elif self.encoder == "graph_emb":
            embs = out["graph_emb"]
        elif self.encoder == "mean":
            embs = (out["language_emb"] + out["graph_emb"])/2
        else:
            raise("Not Implemented")
        logits = self.classifier(embs.float())
        return logits


In [12]:
import torch.nn.functional as F

def evaluate_review_sentiment(model, encoder, finetune=False):
    eval_model = BertGraphMLP(model, encoder, finetune=finetune)

    loss_function = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.877, 1-0.877]).to('cuda'))
    optimizer = torch.optim.Adam(
        [
            {"params":eval_model.classifier.parameters(), "lr": 0.001},       
    ])

    epochs = 2
    device= 'cuda'
    eval_model.to(device)
    if not(finetune):
        eval_model.BertGraph.eval()
    preds = []

    for epoch in range(epochs):
        epoch_loss = 0
        for batch in tqdm.tqdm(train_loader):
            optimizer.zero_grad()  
            embs = eval_model(g.to(device), batch["id"].to(device))  
            loss = loss_function(embs, g.nodes["Review"].data["Positive"][batch["id"]].to(torch.int64).to(device)) / batch_size
            epoch_loss += loss
            loss.backward()
            optimizer.step()    
        print(epoch_loss/len(train_loader))

    eval_model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for batch in tqdm.tqdm(test_loader):
            scores = eval_model(g.to(device), batch["id"].to(device))
            preds.extend(torch.round(F.softmax(scores)[:,1]).to(torch.int64))
            labels.extend(g.nodes["Review"].data["Positive"][batch["id"]].to(torch.int64))

    from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

    preds_cpu = [i.to('cpu') for i in preds]
    labels_cpu = [i.to('cpu') for i in labels]
    print("End of training f1 score accuracy", f1_score(labels_cpu, preds_cpu))
    print("End of training accuracy", accuracy_score(labels_cpu, preds_cpu))
    print("End of training precision", precision_score(labels_cpu, preds_cpu))
    print("End of training recall", recall_score(labels_cpu, preds_cpu))

In [13]:
enc_methods = ["language_emb", "graph_emb", "mean"]

for method in enc_methods:
    print("====== EVALUATING:", method, "======")
    evaluate_review_sentiment(model, method)

====== EVALUATING: language_emb ======


100%|█████████████████████████████████████████████████████████████████████████████| 1606/1606 [08:01<00:00,  3.34it/s]


tensor(0.0139, device='cuda:0', grad_fn=<DivBackward0>)


100%|█████████████████████████████████████████████████████████████████████████████| 1606/1606 [08:03<00:00,  3.32it/s]


tensor(0.0136, device='cuda:0', grad_fn=<DivBackward0>)


  0%|                                                                                        | 0/1606 [00:00<?, ?it/s]/tmp/ipykernel_1680034/98636983.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds.extend(torch.round(F.softmax(scores)[:,1]).to(torch.int64))
100%|█████████████████████████████████████████████████████████████████████████████| 1606/1606 [08:03<00:00,  3.32it/s]


End of training f1 score accuracy 0.934325344691679
End of training accuracy 0.8767453932001038
End of training precision 0.8767453932001038
End of training recall 1.0
====== EVALUATING: graph_emb ======


 73%|███████████████████████████████████████████████████████▉                     | 1167/1606 [05:50<02:12,  3.33it/s]


KeyboardInterrupt: 

In [ ]:
# torch.save(model.state_dict(), "./base_statedict_{}.pt".format(float(val_loss)))

In [10]:
evaluate_review_sentiment(model, 'mean')

100%|█████████████████████████████████████████████████████████████████████████████| 1606/1606 [08:04<00:00,  3.31it/s]


tensor(0.0276, device='cuda:0', grad_fn=<DivBackward0>)


100%|█████████████████████████████████████████████████████████████████████████████| 1606/1606 [08:04<00:00,  3.32it/s]


tensor(0.0274, device='cuda:0', grad_fn=<DivBackward0>)


  0%|                                                                                        | 0/1606 [00:00<?, ?it/s]/tmp/ipykernel_1680034/3680735225.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds.extend(torch.round(F.softmax(scores)[:,1]).to(torch.int64))
100%|█████████████████████████████████████████████████████████████████████████████| 1606/1606 [08:02<00:00,  3.33it/s]


End of training f1 score accuracy 0.7871988781489457
End of training accuracy 0.6691668829483519
End of training precision 0.9026416539050536
End of training recall 0.6979367099849028


In [6]:
model.load_state_dict(torch.load("./base_statedict_5651.253525578349.pt", map_location=torch.device('cuda')))
model.eval()

CLIPGraphModel(
  (graph_model): RGCN(
    (embed): ParameterDict(
        (Brand): Parameter containing: [torch.FloatTensor of size 1900x512]
        (Customer): Parameter containing: [torch.FloatTensor of size 11041x512]
        (Product): Parameter containing: [torch.FloatTensor of size 5334x512]
        (Review): Parameter containing: [torch.FloatTensor of size 77060x512]
    )
    (conv1): HeteroGraphConv(
      (mods): ModuleDict(
        (rev_SOLD_BY): GraphConv(in=512, out=256, normalization=both, activation=None)
        (WROTE): GraphConv(in=512, out=256, normalization=both, activation=None)
        (SOLD_BY): GraphConv(in=512, out=256, normalization=both, activation=None)
        (rev_REVIEW_OF): GraphConv(in=512, out=256, normalization=both, activation=None)
        (REVIEW_OF): GraphConv(in=512, out=256, normalization=both, activation=None)
        (rev_WROTE): GraphConv(in=512, out=256, normalization=both, activation=None)
      )
    )
    (conv2): HeteroGraphConv(
     